In [13]:
#Setup

#common
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import plotly.express as px
import folium

#for ML:
from sklearn.model_selection import train_test_split, KFold, cross_validate, cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
import eli5 # Feature importance evaluation


#set display options
sns.set(style='whitegrid')
pd.set_option('display.max_columns', 36)

#load data
file_path = "/Users/chrisellis/Desktop/CSVs/hotel_bookings.csv"
full_data = pd.read_csv(file_path)

In [3]:
#A bit of data inspection
full_data.head()

,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,babies,meal,country,market_segment,distribution_channel,is_repeated_guest,previous_cancellations,previous_bookings_not_canceled,reserved_room_type,assigned_room_type,booking_changes,deposit_type,agent,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date
0,Resort Hotel,0,342,2015,July,27,1,0,0,2,0.0,0,BB,PRT,Direct,Direct,0,0,0,C,C,3,No Deposit,NaN,NaN,0,Transient,0.0,0,0,Check-Out,2015-07-01
1,Resort Hotel,0,737,2015,July,27,1,0,0,2,0.0,0,BB,PRT,Direct,Direct,0,0,0,C,C,4,No Deposit,NaN,NaN,0,Transient,0.0,0,0,Check-Out,2015-07-01
2,Resort Hotel,0,7,2015,July,27,1,0,1,1,0.0,0,BB,GBR,Direct,Direct,0,0,0,A,C,0,No Deposit,NaN,NaN,0,Transient,75.0,0,0,Check-Out,2015-07-02
3,Resort Hotel,0,13,2015,July,27,1,0,1,1,0.0,0,BB,GBR,Corporate,Corporate,0,0,0,A,A,0,No Deposit,304.0,NaN,0,Transient,75.0,0,0,Check-Out,2015-07-02
4,Resort Hotel,0,14,2015,July,27,1,0,2,2,0.0,0,BB,GBR,Online TA,TA/TO,0,0,0,A,A,0,No Deposit,240.0,NaN,0,Transient,98.0,0,1,Check-Out,2015-07-03


In [4]:
#Checking for missing values
full_data.isnull().sum()

hotel                                  0
is_canceled                            0
lead_time                              0
arrival_date_year                      0
arrival_date_month                     0
arrival_date_week_number               0
arrival_date_day_of_month              0
stays_in_weekend_nights                0
stays_in_week_nights                   0
adults                                 0
children                               4
babies                                 0
meal                                   0
country                              488
market_segment                         0
distribution_channel                   0
is_repeated_guest                      0
previous_cancellations                 0
previous_bookings_not_canceled         0
reserved_room_type                     0
assigned_room_type                     0
booking_changes                        0
deposit_type                           0
agent                              16340
company         

In [5]:
#Replacing missing values:
# agent: If no agency is given, booking was most likely made without one.
# company: If no company is given, it was most likely private.
# country: We'll replace Null with 'Unknown'
# children: If no value is known, we'll replace with 0 (only replacing 4 values shouldn't impact overall findings)
nan_replacements = {'children': 0.0, 'country': 'Unknown', 'agent': 0, 'company': 0}
full_data_cln = full_data.fillna(nan_replacements)

In [6]:
# 'meal' contains values 'Undefined', which is equal to SC.
full_data_cln['meal'].replace('Undefined', 'SC', inplace=True)

#Some of the rows contain 0 adults, 0 children and 0 babies.  We'll drop these entries.

zero_guests = list(full_data_cln.loc[full_data_cln['adults']
                                    + full_data_cln['children']
                                    + full_data_cln['babies']==0].index)
full_data_cln.drop(full_data_cln.index[zero_guests], inplace=True)


In [7]:
#How much data is left?
full_data_cln.shape

(119210, 32)

In [15]:
#After cleaning, we want to seperate Resort and City hotel bookings
#We only want to include bookings that WERE NOT canceled
resort_hotel = full_data_cln.loc[(full_data_cln.hotel == 'Resort Hotel') & (full_data_cln.is_canceled == 0)]
city_hotel = full_data_cln.loc[(full_data_cln.hotel == 'City Hotel') & (full_data_cln.is_canceled == 0)] 

In [52]:
#create pandas dataframe of # of guests/country
country_data = pd.DataFrame(full_data_cln.loc[full_data_cln['is_canceled'] == 0]['country'].value_counts())

#rename column from 'country' to 'number of guests'
country_data.rename(columns={'country': 'number_of_guests'}, inplace=True)
total_guests = country_data.sum()

#create a percentage column, and a country column using indexes
country_data['guests_in_%'] = round(country_data / total_guests*100,2)
country_data['country'] = country_data.index

#pie plot
fig = px.pie(
country_data,
values = 'number_of_guests',
names = 'country',
title = 'Home Country of Guests',
template = 'seaborn')

fig.update_traces(textposition='inside', textinfo='value+percent+label')
fig.show()


#For easier viz, combine all countries with less than 2 guests into a new grouping named 'Other'

country_data.loc[country_data['guests_in_%']<2, 'country'] = 'Other'

fig = px.pie(
country_data,
values = 'number_of_guests',
names = 'country',
title = 'Home Country of Guests',
template = 'seaborn')

fig.update_traces(textposition='inside', textinfo='value+percent+label')
fig.show()



In [11]:
#getting number of actual guests by country
#start by populating a new DataFrame with aggregate count
country_data = pd.DataFrame(full_data_cln.loc[full_data_cln.is_canceled == 0].country.value_counts())
country_data.rename(columns={
    'country': 'Total Guests Per Country'
}, inplace=True)
total_guests = country_data['Total Guests Per Country'].sum()
country_data['Guests as %'] = round(country_data['Total Guests Per Country'] / total_guests * 100, 2)
country_data['country'] = country_data.index

#viz of country data

guest_map = px.choropleth(
country_data,
locations = country_data.index,
color = country_data['Guests as %'],
hover_name = country_data.index,
color_continuous_scale = 'Viridis',
title = 'Home Country of Guests')

guest_map.show()

In [ ]:
#People from all over the world are staying in these hotels, however most guests are from Portugal and surrounding European countries

In [64]:
#How much are guests paying for rooms?
# EUR currencey is assumed
# Babies are part of our dataset, but we only want to include adults and children in our rate per person calculation.


resort_hotel['adr_pp'] = round(resort_hotel['adr']/(resort_hotel['adults'] + resort_hotel['children']),2)
city_hotel['adr_pp'] = round(city_hotel['adr']/(city_hotel['adults'] + city_hotel['children']),2)


#print mean price per guest for city hotel and resort hotel

print("""From all non-canceled bookings, across all room types and meals, the average prices are:
Resort hotel: {:.2f} € per night and person.
City hotel: {:.2f} € per night and person."""
      .format(resort_hotel["adr_pp"].mean(), city_hotel["adr_pp"].mean()))


From all non-canceled bookings, across all room types and meals, the average prices are:
Resort hotel: 47.49 € per night and person.
City hotel: 59.27 € per night and person.


/var/folders/vn/mq2z43s14wlgk7_snr2rtwmh0000gn/T/ipykernel_46898/463084217.py:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/vn/mq2z43s14wlgk7_snr2rtwmh0000gn/T/ipykernel_46898/463084217.py:7: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

